# Package

In [1]:
# package
import pandas as pd
import re
import phonenumbers
from phonenumbers import NumberParseException, PhoneNumberFormat

# Import dataset

In [2]:
# Import dataset
df = pd.read_csv("sales_data_sample.csv", encoding="latin1")

# Rename column headers

In [3]:
df.columns

Index(['ORDERNUMBER', 'QUANTITYORDERED', 'PRICEEACH', 'ORDERLINENUMBER',
       'SALES', 'ORDERDATE', 'STATUS', 'QTR_ID', 'MONTH_ID', 'YEAR_ID',
       'PRODUCTLINE', 'MSRP', 'PRODUCTCODE', 'CUSTOMERNAME', 'PHONE',
       'ADDRESSLINE1', 'ADDRESSLINE2', 'CITY', 'STATE', 'POSTALCODE',
       'COUNTRY', 'TERRITORY', 'CONTACTLASTNAME', 'CONTACTFIRSTNAME',
       'DEALSIZE'],
      dtype='object')

In [4]:
# 1. Standardize column names (lowercase, no spaces)
df.columns = (df.columns.str.strip().str.lower().str.replace(" ", "_"))

In [5]:
# make more readable
rename_dict = {
    "ordernumber": "order_number",
    "quantityordered": "quantity_ordered",
    "priceeach": "price_each",
    "orderlinenumber": "order_line_number",
    "sales": "sales",
    "orderdate": "order_date",
    "status": "status",
    "qtr_id": "quarter_id",
    "month_id": "month_id",
    "year_id": "year_id",
    "productline": "product_line",
    "msrp": "msrp",
    "productcode": "product_code",
    "customername": "customer_name",
    "phone": "phone",
    "addressline1": "address_line1",
    "addressline2": "address_line2",
    "city": "city",
    "state": "state",
    "postalcode": "postal_code",
    "country": "country",
    "territory": "territory",
    "contactlastname": "contact_last_name",
    "contactfirstname": "contact_first_name",
    "dealsize": "deal_size"
}

# Apply rename
df = df.rename(columns=rename_dict)

In [6]:
df.columns

Index(['order_number', 'quantity_ordered', 'price_each', 'order_line_number',
       'sales', 'order_date', 'status', 'quarter_id', 'month_id', 'year_id',
       'product_line', 'msrp', 'product_code', 'customer_name', 'phone',
       'address_line1', 'address_line2', 'city', 'state', 'postal_code',
       'country', 'territory', 'contact_last_name', 'contact_first_name',
       'deal_size'],
      dtype='object')

# Removing duplicate

In [7]:
#Check for duplicate rows
duplicate_rows = df.duplicated()

# Count of duplicate rows
print(f"Number of duplicate rows: {duplicate_rows.sum()}")

print("Shape of DataFrame Before Removing Duplicates: ", df.shape)

# Drop the duplicates
df = df.drop_duplicates()

# Checking the shape of the data after dropping duplicates
print("Shape of DataFrame After Removing Duplicates: ", df.shape)

Number of duplicate rows: 0
Shape of DataFrame Before Removing Duplicates:  (2823, 25)
Shape of DataFrame After Removing Duplicates:  (2823, 25)


# Standardize values 

In [8]:
#Standardize text fields
df["status"] = df["status"].str.strip().str.upper()
df["country"] = df["country"].str.strip().str.title()
df["state"] = df["state"].fillna("").str.upper()
df["city"] = df["city"].str.title()

In [9]:
# convert datatype year/month/quarter to integers
df["year_id"] = df["year_id"].astype(int)
df["month_id"] = df["month_id"].astype(int)
df["quarter_id"] = df["quarter_id"].astype(int)

# Convert date formats

In [10]:
#Convert date formats
df["order_date"] = pd.to_datetime(df["order_date"], errors="coerce", dayfirst=True)
df["order_date"] = df["order_date"].dt.strftime("%d-%m-%Y")

C:\Users\Kamalachandran\AppData\Local\Temp\ipykernel_7372\4063435976.py:2: UserWarning: Parsing dates in %m/%d/%Y %H:%M format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df["order_date"] = pd.to_datetime(df["order_date"], errors="coerce", dayfirst=True)


# Check and fix data types

In [11]:
# Convert data types
df["order_number"] = df["order_number"].astype(int)
df["quantity_ordered"] = df["quantity_ordered"].astype(int)
df["order_line_number"] = df["order_line_number"].astype(int)

df["price_each"] = df["price_each"].astype(float)
df["sales"] = df["sales"].astype(float)

df["postal_code"] = df["postal_code"].astype(str)

In [12]:
#Trim whitespace in names
df["customer_name"] = df["customer_name"].str.strip().str.title()
df["contact_last_name"] = df["contact_last_name"].str.strip().str.title()
df["contact_first_name"] = df["contact_first_name"].str.strip().str.title()

# Format phone number

In [13]:
df['phone'] = df['phone'].str.replace(r"[.\s]", "", regex=True)
df['phone'].head()

0      2125557818
1        26471555
2    +33146627555
3      6265557265
4      6505551386
Name: phone, dtype: object

In [14]:
def format_phone_number(number, default_region='US'):
    try:
        number = number.replace(".", "").replace("(", "").replace(")", "").replace(" ", "")
        parsed = phonenumbers.parse(str(number), default_region)
        if phonenumbers.is_valid_number(parsed):
            return str(parsed.national_number)
        else:
            return None
    except NumberParseException:
        return None

# -----------------------------
# Step 3: Apply formatting
# -----------------------------
df['phone'] = df['phone'].apply(format_phone_number)

# Handling missing values

In [15]:
df.isnull().sum()

order_number             0
quantity_ordered         0
price_each               0
order_line_number        0
sales                    0
order_date               0
status                   0
quarter_id               0
month_id                 0
year_id                  0
product_line             0
msrp                     0
product_code             0
customer_name            0
phone                 1483
address_line1            0
address_line2         2521
city                     0
state                    0
postal_code              0
country                  0
territory             1074
contact_last_name        0
contact_first_name       0
deal_size                0
dtype: int64

In [16]:
df["address_line2"] = df["address_line2"].fillna("NA")
df["territory"] = df["territory"].fillna("NA")

df["state"]=df["state"].replace("",pd.NA)
df["state"]=df["state"].replace(pd.NA,"Unknown")

df["phone"]=df["phone"].replace("",pd.NA)
df["phone"]=df["phone"].replace(pd.NA,"Unknown")

df["postal_code"] = df["postal_code"].replace("",pd.NA)
df["postal_code"] = df["postal_code"].replace("nan",pd.NA)
df["postal_code"] = df["postal_code"].replace(pd.NA,"Unknown")

df.isnull().sum()

order_number          0
quantity_ordered      0
price_each            0
order_line_number     0
sales                 0
order_date            0
status                0
quarter_id            0
month_id              0
year_id               0
product_line          0
msrp                  0
product_code          0
customer_name         0
phone                 0
address_line1         0
address_line2         0
city                  0
state                 0
postal_code           0
country               0
territory             0
contact_last_name     0
contact_first_name    0
deal_size             0
dtype: int64

# Export dataset

In [17]:
# Export dataset
df.to_csv("cleaned_data.csv", index=False, encoding="utf-8")